In [1]:
import numpy as np
import time
import os

from opfython.models.unsupervised import UnsupervisedOPF
from opfython.models.supervised import SupervisedOPF
from sklearn.metrics import accuracy_score, recall_score, f1_score

import matplotlib.pyplot as plt

opfSup = SupervisedOPF(distance='log_squared_euclidean', pre_computed_distance=None)

2020-02-27 18:42:43,134 - opfython.models.supervised — INFO — Overriding class: OPF -> SupervisedOPF.
2020-02-27 18:42:43,198 - opfython.core.opf — INFO — Creating class: OPF.
2020-02-27 18:42:43,201 - opfython.core.opf — DEBUG — Distance: log_squared_euclidean | Pre-computed distance: False.
2020-02-27 18:42:43,203 - opfython.core.opf — INFO — Class created.
2020-02-27 18:42:43,204 - opfython.models.supervised — INFO — Class overrided.


In [2]:
def getMeanCov(kmax, data, labels):
    
    
    classes = np.unique(labels)

    max_class = 0
    max_class_idx = -1
    dt = []
    lbl = []

    for c in range(len(classes)):
        idx = labels == classes[c]
        dt.append(data[idx].copy())
        lbl.append(labels[idx].copy())
        if max_class<len(dt[c]):
            max_class_idx = c
            max_class = len(dt[c]) 
    
    
    
    out = {}

    for c in range(len(classes)):
        if c == max_class_idx:
            out[c] = None
            continue

        print("=============================")    
        print("Category:", c)
        train_data = dt[c]
        train_label = lbl[c]

        print("Kmax:", kmax)

        
        opf = UnsupervisedOPF(min_k=1, max_k=kmax, distance='log_squared_euclidean', pre_computed_distance=None)
        
        opf.fit(train_data,train_label)
        preds=[]
        for i in range((opf.subgraph.n_nodes)):
            preds.append(opf.subgraph.nodes[i].cluster_label)
        #print(preds)

        print("k:", opf.subgraph.best_k)
        print("N# clusters", opf.subgraph.n_clusters)
        print("\n")

        c_map = {}
        preds = np.asarray(preds)

        for cl in range(opf.subgraph.n_clusters):
            idx_c = preds == cl
            cluster = train_data[idx_c]
            mean = cluster.mean(0)
            cov = np.cov(cluster, rowvar=False)

            if cov.size!=1:
                c_map[cl] = (mean, cov, len(cluster))
            else:
                c_map[cl] = None
        out[c] = (c_map, opf.subgraph.n_clusters,len(train_label) )
    return out, len(classes), classes, max_class

In [3]:
def generateSamples(n_samples, mean, cov):
    # Define epsilon. Avoid cov decay very rapidly and unstabilize the Cholesky decomposition
    epsilon = 0.0001
    #dimension
    d = len(mean)
    K = cov + epsilon*np.identity(d)
    
    z = np.random.multivariate_normal(mean=mean.reshape(d,), cov=K, size=n_samples)

    return z

In [4]:
def O2PF(k,f, p,data, labels,trainSet):
    start_time = time.time()
        
    values, n_classes, classes, max_class = getMeanCov(k,data,labels)


    finalSamples = []  

    for i in range(n_classes):
        if values[i] is None:
            continue
        values_ = values[i][0]
        n_clusters = values[i][1]
        classe = classes[i]
        newSamples = []


        #number of samples of class classe
        samplesPerClass = values[i][2]  
        #total number of samples to be generated to class classe
        NumberOfSamples = max_class-samplesPerClass

        #number of samples that will be actually generated to class classe
        NumberOfSamplesToGenerate = int(NumberOfSamples*p)


        for j in range(n_clusters):               
            if values_[j] is None:
                continue
            mean = values_[j][0]
            cov = values_[j][1]
            #number of samples from cluster j
            n_samples = values_[j][2]  

            #number of samples generated based on cluster j
            samplesToGenerate = int((NumberOfSamplesToGenerate* n_samples)/samplesPerClass)                

            newSamples.append(generateSamples(samplesToGenerate, mean,cov))
        
        if len(newSamples[0])==0:
            continue
        
#         print(len(newSamples[0]))
#         print(newSamples[0])
#         print(newSamples)
        newSamples = np.asarray(np.vstack(newSamples))
        newSamples = np.insert(newSamples,len(newSamples[0]),classe , axis=1)            

        finalSamples.append(newSamples)
        
    finalSamples.append(trainSet)
    finalSamples = np.asarray(np.vstack(finalSamples))


        
    end_time = time.time()
    return finalSamples[:,:-1], finalSamples[:,-1].astype(np.int)  , end_time-start_time

In [5]:
def classify(x_train,y_train, x_valid, y_valid):
    # Training the OPF                
    opfSup.fit(x_train, y_train)

    # Prediction of the validation samples
    y_pred = opfSup.predict(x_valid)
    y_pred = np.array(y_pred)

    # Validation measures for this k nearest neighbors
    accuracy = accuracy_score(y_valid, y_pred)
    recall = recall_score(y_valid, y_pred, pos_label=2) # assuming that 2 is the minority class
    f1 = f1_score(y_valid, y_pred, pos_label=2)
    return accuracy, recall, f1

# Validação

In [6]:
#datasets = ['cervical_cancer','mammographic','messidor','original', 'diagnostic', 'prognostic']
datasets = [ 'messidor']
folds = np.arange(3,21)
percents = [0.25, 0.5,0.75,1.0]
#percents = [ 0.5,0.75,1.0]
#percents = [0.25]
#k_max = [5,10,30,50,75,100]
k_max = [5,10,20,30,40,50,60,70,80,90,100]

# datasets = ['original']
# folds = np.arange(1,2)
# percents = [0.5]
# k_max = [5]

for dsds in range(len(datasets)):
    ds = datasets[dsds]
    
    for ff in range(len(folds)): 
        f = folds[ff]
        
        train = np.loadtxt('data/{}/{}/train.txt'.format(ds,f),delimiter=',', dtype=np.float32)
        valid = np.loadtxt('data/{}/{}/valid.txt'.format(ds,f),delimiter=',', dtype=np.float32)
        test = np.loadtxt('data/{}/{}/test.txt'.format(ds,f),delimiter=',', dtype=np.float32)
        data = train[:,:-1]
        labels = train[:,-1].astype(np.int)  
        fmt = '%.5f,'*(len(train[0])-1)+'%.d'

        x_valid = valid[:,:-1]
        y_valid = valid[:,-1].astype(np.int)  

        x_test = test[:,:-1]
        y_test = test[:,-1].astype(np.int)  

        for pp in range(len(percents)): 
            p = percents[pp]
            
            
            
            path = 'Results/OPF/{}/{}/{}'.format(ds,f,p)
            if not os.path.exists(path):
                os.makedirs(path)
            
            #results_print = open('{}/{}.txt'.format(path,'results'), "w")
            #validation_print = open('{}/{}.txt'.format(path,'validation'), "w")
            validation_print=[]
            results_print=[]
            #strResults = ''
            #strValidation = ''
            
            # validation
            k_best = 0
            best_recall = 0
            for kk in range(len(k_max)): 
                k = k_max[kk]
                x_train, y_train, elapsed_time = O2PF(k,f, p,data,labels, train)                 
                accuracy, recall, f1 = classify(x_train,y_train, x_valid, y_valid)
                

                validation_print.append([k, accuracy, recall, f1,elapsed_time])
                
                
                if recall>best_recall:
                    best_recall = recall
                    k_best = k
            
            # test, 0.5,0.75,1.0
            x_train, y_train, elapsed_time = O2PF(k_best,f, p,data,labels, train)  
            accuracy, recall, f1 = classify(x_train,y_train, x_test, y_test)
            

            results_print.append([k_best,accuracy, recall, f1,elapsed_time])
            
            #print('{} {:d} {:.5f} {:.5f} {:.5f} {:.5f} \n'.format('results',k_best,accuracy, recall, f1,elapsed_time))
            
            
            np.savetxt('{}/{}.txt'.format(path,'validation'), validation_print, fmt='%d,%.5f,%.5f,%.5f,%.5f')
            #validation_print.write(strValidation)
            np.savetxt('{}/{}.txt'.format(path,'results'), results_print, fmt='%d,%.5f,%.5f,%.5f,%.5f')
            #results_print.write('{:d} {:.5f} {:.5f} {:.5f} {:.5f} \n'.format(k_best, accuracy, recall, f1,elapsed_time))


Category: 0
Kmax: 5
2020-02-27 18:43:05,873 - opfython.models.unsupervised — INFO — Overriding class: OPF -> UnsupervisedOPF.
2020-02-27 18:43:05,874 - opfython.core.opf — INFO — Creating class: OPF.
2020-02-27 18:43:05,874 - opfython.core.opf — DEBUG — Distance: log_squared_euclidean | Pre-computed distance: False.
2020-02-27 18:43:05,875 - opfython.core.opf — INFO — Class created.
2020-02-27 18:43:05,875 - opfython.models.unsupervised — INFO — Class overrided.
2020-02-27 18:43:05,876 - opfython.models.unsupervised — INFO — Clustering with classifier ...
2020-02-27 18:43:05,878 - opfython.models.unsupervised — DEBUG — Calculating the best minimum cut within [1, 5] ...
2020-02-27 18:43:09,223 - opfython.models.unsupervised — DEBUG — Best: 5 | Minimum cut: 35.89010324241058.
2020-02-27 18:43:09,244 - opfython.models.unsupervised — INFO — Classifier has been clustered with.
2020-02-27 18:43:09,244 - opfython.models.unsupervised — INFO — Number of clusters: 64.
2020-02-27 18:43:09,245 - o

2020-02-27 18:43:29,260 - opfython.models.supervised — INFO — Prediction time: 1.6113061904907227 seconds.
Category: 0
Kmax: 20
2020-02-27 18:43:29,262 - opfython.models.unsupervised — INFO — Overriding class: OPF -> UnsupervisedOPF.
2020-02-27 18:43:29,263 - opfython.core.opf — INFO — Creating class: OPF.
2020-02-27 18:43:29,263 - opfython.core.opf — DEBUG — Distance: log_squared_euclidean | Pre-computed distance: False.
2020-02-27 18:43:29,265 - opfython.core.opf — INFO — Class created.
2020-02-27 18:43:29,265 - opfython.models.unsupervised — INFO — Class overrided.
2020-02-27 18:43:29,266 - opfython.models.unsupervised — INFO — Clustering with classifier ...
2020-02-27 18:43:29,268 - opfython.models.unsupervised — DEBUG — Calculating the best minimum cut within [1, 20] ...
2020-02-27 18:43:35,557 - opfython.models.unsupervised — DEBUG — Best: 19 | Minimum cut: 5.897650835472145.
2020-02-27 18:43:35,586 - opfython.models.unsupervised — INFO — Classifier has been clustered with.
2020-

2020-02-27 18:49:34,714 - opfython.models.supervised — INFO — Data has been predicted.
2020-02-27 18:49:34,717 - opfython.models.supervised — INFO — Prediction time: 14.943298578262329 seconds.
Category: 0
Kmax: 40
2020-02-27 18:49:34,731 - opfython.models.unsupervised — INFO — Overriding class: OPF -> UnsupervisedOPF.
2020-02-27 18:49:34,733 - opfython.core.opf — INFO — Creating class: OPF.
2020-02-27 18:49:34,735 - opfython.core.opf — DEBUG — Distance: log_squared_euclidean | Pre-computed distance: False.
2020-02-27 18:49:34,737 - opfython.core.opf — INFO — Class created.
2020-02-27 18:49:34,740 - opfython.models.unsupervised — INFO — Class overrided.
2020-02-27 18:49:34,747 - opfython.models.unsupervised — INFO — Clustering with classifier ...
2020-02-27 18:49:34,823 - opfython.models.unsupervised — DEBUG — Calculating the best minimum cut within [1, 40] ...
2020-02-27 18:52:58,820 - opfython.models.unsupervised — DEBUG — Best: 28 | Minimum cut: 1.3816067665287082.
2020-02-27 18:52:

2020-02-27 18:55:19,299 - opfython.models.supervised — INFO — Predicting data ...
2020-02-27 18:55:25,382 - opfython.models.supervised — INFO — Data has been predicted.
2020-02-27 18:55:25,383 - opfython.models.supervised — INFO — Prediction time: 6.081863880157471 seconds.
Category: 0
Kmax: 60
2020-02-27 18:55:25,393 - opfython.models.unsupervised — INFO — Overriding class: OPF -> UnsupervisedOPF.
2020-02-27 18:55:25,395 - opfython.core.opf — INFO — Creating class: OPF.
2020-02-27 18:55:25,398 - opfython.core.opf — DEBUG — Distance: log_squared_euclidean | Pre-computed distance: False.
2020-02-27 18:55:25,401 - opfython.core.opf — INFO — Class created.
2020-02-27 18:55:25,404 - opfython.models.unsupervised — INFO — Class overrided.
2020-02-27 18:55:25,408 - opfython.models.unsupervised — INFO — Clustering with classifier ...
2020-02-27 18:55:25,423 - opfython.models.unsupervised — DEBUG — Calculating the best minimum cut within [1, 60] ...
2020-02-27 18:57:10,273 - opfython.models.uns

2020-02-27 19:00:09,022 - opfython.models.supervised — INFO — Training time: 22.824754238128662 seconds.
2020-02-27 19:00:09,023 - opfython.models.supervised — INFO — Predicting data ...
2020-02-27 19:00:15,266 - opfython.models.supervised — INFO — Data has been predicted.
2020-02-27 19:00:15,268 - opfython.models.supervised — INFO — Prediction time: 6.242661714553833 seconds.
Category: 0
Kmax: 80
2020-02-27 19:00:15,276 - opfython.models.unsupervised — INFO — Overriding class: OPF -> UnsupervisedOPF.
2020-02-27 19:00:15,278 - opfython.core.opf — INFO — Creating class: OPF.
2020-02-27 19:00:15,279 - opfython.core.opf — DEBUG — Distance: log_squared_euclidean | Pre-computed distance: False.
2020-02-27 19:00:15,282 - opfython.core.opf — INFO — Class created.
2020-02-27 19:00:15,284 - opfython.models.unsupervised — INFO — Class overrided.
2020-02-27 19:00:15,286 - opfython.models.unsupervised — INFO — Clustering with classifier ...
2020-02-27 19:00:15,299 - opfython.models.unsupervised — 

...

2020-02-29 01:15:42,000 - opfython.models.supervised — INFO — Classifier has been fitted.
2020-02-29 01:15:42,002 - opfython.models.supervised — INFO — Training time: 28.167612314224243 seconds.
2020-02-29 01:15:42,003 - opfython.models.supervised — INFO — Predicting data ...
2020-02-29 01:15:49,203 - opfython.models.supervised — INFO — Data has been predicted.
2020-02-29 01:15:49,205 - opfython.models.supervised — INFO — Prediction time: 7.198751211166382 seconds.
Category: 0
Kmax: 40
2020-02-29 01:15:49,217 - opfython.models.unsupervised — INFO — Overriding class: OPF -> UnsupervisedOPF.
2020-02-29 01:15:49,220 - opfython.core.opf — INFO — Creating class: OPF.
2020-02-29 01:15:49,222 - opfython.core.opf — DEBUG — Distance: log_squared_euclidean | Pre-computed distance: False.
2020-02-29 01:15:49,224 - opfython.core.opf — INFO — Class created.
2020-02-29 01:15:49,226 - opfython.models.unsupervised — INFO — Class overrided.
2020-02-29 01:15:49,228 - opfython.models.unsupervised — INFO 

2020-02-29 01:19:08,537 - opfython.models.supervised — INFO — Classifier has been fitted.
2020-02-29 01:19:08,539 - opfython.models.supervised — INFO — Training time: 25.38250207901001 seconds.
2020-02-29 01:19:08,540 - opfython.models.supervised — INFO — Predicting data ...
2020-02-29 01:19:15,401 - opfython.models.supervised — INFO — Data has been predicted.
2020-02-29 01:19:15,402 - opfython.models.supervised — INFO — Prediction time: 6.859145641326904 seconds.
Category: 0
Kmax: 60
2020-02-29 01:19:15,412 - opfython.models.unsupervised — INFO — Overriding class: OPF -> UnsupervisedOPF.
2020-02-29 01:19:15,414 - opfython.core.opf — INFO — Creating class: OPF.
2020-02-29 01:19:15,416 - opfython.core.opf — DEBUG — Distance: log_squared_euclidean | Pre-computed distance: False.
2020-02-29 01:19:15,419 - opfython.core.opf — INFO — Class created.
2020-02-29 01:19:15,420 - opfython.models.unsupervised — INFO — Class overrided.
2020-02-29 01:19:15,423 - opfython.models.unsupervised — INFO —

2020-02-29 01:23:58,116 - opfython.models.supervised — INFO — Classifier has been fitted.
2020-02-29 01:23:58,118 - opfython.models.supervised — INFO — Training time: 25.947803258895874 seconds.
2020-02-29 01:23:58,119 - opfython.models.supervised — INFO — Predicting data ...
2020-02-29 01:24:05,323 - opfython.models.supervised — INFO — Data has been predicted.
2020-02-29 01:24:05,324 - opfython.models.supervised — INFO — Prediction time: 7.202373743057251 seconds.
Category: 0
Kmax: 80
2020-02-29 01:24:05,333 - opfython.models.unsupervised — INFO — Overriding class: OPF -> UnsupervisedOPF.
2020-02-29 01:24:05,335 - opfython.core.opf — INFO — Creating class: OPF.
2020-02-29 01:24:05,339 - opfython.core.opf — DEBUG — Distance: log_squared_euclidean | Pre-computed distance: False.
2020-02-29 01:24:05,341 - opfython.core.opf — INFO — Class created.
2020-02-29 01:24:05,343 - opfython.models.unsupervised — INFO — Class overrided.
2020-02-29 01:24:05,346 - opfython.models.unsupervised — INFO 

2020-02-29 01:31:26,064 - opfython.models.supervised — INFO — Classifier has been fitted.
2020-02-29 01:31:26,066 - opfython.models.supervised — INFO — Training time: 24.661345958709717 seconds.
2020-02-29 01:31:26,067 - opfython.models.supervised — INFO — Predicting data ...
2020-02-29 01:31:32,937 - opfython.models.supervised — INFO — Data has been predicted.
2020-02-29 01:31:32,938 - opfython.models.supervised — INFO — Prediction time: 6.86830997467041 seconds.
Category: 0
Kmax: 100
2020-02-29 01:31:32,946 - opfython.models.unsupervised — INFO — Overriding class: OPF -> UnsupervisedOPF.
2020-02-29 01:31:32,947 - opfython.core.opf — INFO — Creating class: OPF.
2020-02-29 01:31:32,949 - opfython.core.opf — DEBUG — Distance: log_squared_euclidean | Pre-computed distance: False.
2020-02-29 01:31:32,950 - opfython.core.opf — INFO — Class created.
2020-02-29 01:31:32,952 - opfython.models.unsupervised — INFO — Class overrided.
2020-02-29 01:31:32,956 - opfython.models.unsupervised — INFO 

2020-02-29 01:37:42,666 - opfython.models.supervised — INFO — Classifier has been fitted.
2020-02-29 01:37:42,668 - opfython.models.supervised — INFO — Training time: 26.059152603149414 seconds.
2020-02-29 01:37:42,669 - opfython.models.supervised — INFO — Predicting data ...
2020-02-29 01:37:50,028 - opfython.models.supervised — INFO — Data has been predicted.
2020-02-29 01:37:50,030 - opfython.models.supervised — INFO — Prediction time: 7.357891082763672 seconds.


In [ ]:
def plotValidation(y):
    y = [1, 4, 16, 32]
    plt.plot(y, 'r*-', linewidth=3, markersize=10)
    # or (a bit easier to read)
    plt.plot(y, color='r', linestyle='solid', marker='*', linewidth=3, markersize=10)